<a href="https://colab.research.google.com/github/BreakoutMentors/Data-Science-and-Machine-Learning/blob/main/machine_learning/lesson%204%20-%20ML%20Apps/Gradio/ML_Object_Recognition_using_Pretrained_Transformers_Gradio_App_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install    \
    'beautifulsoup4==4.9.3' \
    'bs4==0.0.1'    \
    'requests-file==1.5.1'  \
    'torch>=1.10.1'    \
    'transformers==4.26.0'   \
    'validators==0.18.2'    \
    'timm>=0.5.4'   \
    'gradio'


In [2]:
import io
import gradio as gr
import matplotlib.pyplot as plt
import requests, validators
import torch
import pathlib
from PIL import Image
from transformers import AutoFeatureExtractor, DetrForObjectDetection, YolosForObjectDetection

import os

In [4]:
# colors for visualization
COLORS = [
    [0.000, 0.447, 0.741],
    [0.850, 0.325, 0.098],
    [0.929, 0.694, 0.125],
    [0.494, 0.184, 0.556],
    [0.466, 0.674, 0.188],
    [0.301, 0.745, 0.933]
]

def make_prediction(img, feature_extractor, model):
    inputs = feature_extractor(img, return_tensors="pt")
    outputs = model(**inputs)
    img_size = torch.tensor([tuple(reversed(img.size))])
    processed_outputs = feature_extractor.post_process(outputs, img_size)
    return processed_outputs[0]

def fig2img(fig):
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img


def visualize_prediction(pil_img, output_dict, threshold=0.7, id2label=None):
    keep = output_dict["scores"] > threshold
    boxes = output_dict["boxes"][keep].tolist()
    scores = output_dict["scores"][keep].tolist()
    labels = output_dict["labels"][keep].tolist()
    if id2label is not None:
        labels = [id2label[x] for x in labels]

    plt.figure(figsize=(16, 10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for score, (xmin, ymin, xmax, ymax), label, color in zip(scores, boxes, labels, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color=color, linewidth=3))
        ax.text(xmin, ymin, f"{label}: {score:0.2f}", fontsize=15, bbox=dict(facecolor="yellow", alpha=0.5))
    plt.axis("off")
    return fig2img(plt.gcf())


def detect_objects(model_name,url_input,image_input,threshold):
    
    #Extract model and feature extractor
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
    
    if 'detr' in model_name:
        
        model = DetrForObjectDetection.from_pretrained(model_name)
        
    elif 'yolos' in model_name:
    
        model = YolosForObjectDetection.from_pretrained(model_name)
    
    if validators.url(url_input):
        image = Image.open(requests.get(url_input, stream=True).raw)
        
    elif image_input:
        image = image_input
    
    #Make prediction
    processed_outputs = make_prediction(image, feature_extractor, model)
    
    #Visualize prediction
    viz_img = visualize_prediction(image, processed_outputs, threshold, model.config.id2label)
    
    return viz_img   
        
def set_example_image(example: list) -> dict:
    return gr.Image.update(value=example[0])

def set_example_url(example: list) -> dict:
    return gr.Textbox.update(value=example[0])


In [5]:
title = """<h1 id="title">Object Detection App with DETR and YOLOS</h1>"""

description = """
Links to HuggingFace Models:
- [facebook/detr-resnet-50](https://huggingface.co/facebook/detr-resnet-50)  
- [hustvl/yolos-small](https://huggingface.co/hustvl/yolos-small)
- [hustvl/yolos-tiny](https://huggingface.co/hustvl/yolos-tiny)
"""

models = ["facebook/detr-resnet-50",'hustvl/yolos-small','hustvl/yolos-tiny']
urls = ["https://c8.alamy.com/comp/J2AB4K/the-new-york-stock-exchange-on-the-wall-street-in-new-york-J2AB4K.jpg"]

twitter_link = """
[![](https://img.shields.io/twitter/follow/kaithedataguy?label=@kaithedataguy&style=social)](https://twitter.com/kaithedataguy)
"""

css = '''
h1#title {
  text-align: center;
}
'''
demo = gr.Blocks(css=css)

with demo:
    gr.Markdown(title)
    gr.Markdown(description)
    gr.Markdown(twitter_link)
    options = gr.Dropdown(choices=models, label='Select Object Detection Model', show_label=True)
    slider_input = gr.Slider(minimum=0.2, maximum=1, value=0.7, label='Prediction Threshold')
    
    with gr.Tabs():
        with gr.TabItem('Image URL'):
            with gr.Row():
                url_input = gr.Textbox(lines=2,label='Enter valid image URL here..')
                img_output_from_url = gr.Image(shape=(650,650))
                
            with gr.Row():
                example_url = gr.Dataset(components=[url_input],samples=[[str(url)] for url in urls])
            
            url_but = gr.Button('Detect')
     
        with gr.TabItem('Image Upload'):
            with gr.Row():
                img_input = gr.Image(type='pil')
                img_output_from_upload= gr.Image(shape=(650,650))
                
            with gr.Row(): 
                example_images = gr.Dataset(components=[img_input],
                                            samples=[[path.as_posix()]
                                                 for path in sorted(pathlib.Path('images').rglob('*.JPG'))])
            img_but = gr.Button('Detect')

        with gr.TabItem('Webcam'):
            with gr.Row():
                web_input = gr.Image(source='webcam', type='pil')
                img_output_from_web = gr.Image(shape=(650,650))
                
            web_but = gr.Button('Detect')
        
    
    url_but.click(detect_objects, inputs=[options, url_input, img_input, slider_input], outputs=img_output_from_url, queue=True)
    img_but.click(detect_objects, inputs=[options, url_input, img_input, slider_input], outputs=img_output_from_upload, queue=True)
    web_but.click(detect_objects, inputs=[options,url_input, web_input, slider_input], outputs=img_output_from_web, queue=True)
    example_images.click(fn=set_example_image, inputs=[example_images], outputs=[img_input])
    example_url.click(fn=set_example_url,inputs=[example_url], outputs=[url_input])
    
demo.launch(enable_queue=True, debug=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://94daa643-dd89-4ca8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
